# Projecting Reviews about the Host per Listing, per Year Based on the Topics

In [1]:

# import and setup modules we'll be using in this notebook
import logging
import itertools
import pandas as pd
import numpy as np
import seaborn as sns
import gensim
import sys
reload(sys)  
import matplotlib.ticker as mtick
import re
import cPickle
sys.setdefaultencoding('utf8')
logging.basicConfig(format='%(levelname)s : %(message)s', level=logging.INFO)
logging.root.level = logging.INFO  # ipython sometimes messes up the logging setup; restore
from gensim.utils import smart_open, simple_preprocess
from gensim.corpora.wikicorpus import _extract_pages, filter_wiki
from gensim.parsing.preprocessing import STOPWORDS
import matplotlib.pylab as plt
%matplotlib inline

# Reading in vectorized corpus

In [2]:
with open(r"corpus.pickle", "rb") as input_file:
    corpus= cPickle.load(input_file)

In [3]:
model = gensim.models.LdaModel.load('lda1.pkl')
#top_topics = model.top_topics(corpus, num_words=5)
model.show_topics()
# Average topic coherence is the sum of topic coherences of all topics, divided by the number of topics.
#avg_topic_coherence = sum([t[1] for t in top_topics]) / num_topics
#print('Average topic coherence: %.4f.' % avg_topic_coherence)

[(0,
  u'0.039*"arrived" + 0.031*"late" + 0.025*"flight" + 0.018*"arrival" + 0.017*"morning" + 0.017*"luggage" + 0.016*"greeted" + 0.014*"left" + 0.014*"breakfast" + 0.013*"early"'),
 (1,
  u'0.161*"nyc" + 0.033*"alex" + 0.027*"chris" + 0.024*"sam" + 0.022*"john" + 0.019*"mike" + 0.018*"en" + 0.018*"exceeded" + 0.017*"anna" + 0.015*"exceeded_expectation"'),
 (2,
  u'0.070*"subway" + 0.053*"close" + 0.034*"walk" + 0.030*"manhattan" + 0.028*"station" + 0.026*"minute" + 0.026*"train" + 0.024*"park" + 0.024*"away" + 0.024*"block"'),
 (3,
  u'0.035*"easy" + 0.026*"check" + 0.026*"helpful" + 0.025*"question" + 0.023*"responsive" + 0.019*"accommodating" + 0.016*"communication" + 0.014*"needed" + 0.014*"meet" + 0.013*"time"'),
 (4,
  u'0.280*"new" + 0.232*"york" + 0.231*"new_york" + 0.020*"city" + 0.010*"trip" + 0.006*"visit" + 0.006*"visiting" + 0.005*"julia" + 0.005*"consider" + 0.005*"jason"'),
 (5,
  u'0.032*"wonderful" + 0.032*"home" + 0.026*"recommend" + 0.025*"feel" + 0.024*"amazing" + 

In [4]:
topics=[]
top_topics=model.show_topics()
for t in top_topics:
    topic={}
    for chunk in t[1].split('+'):
        topic[chunk.split('*')[1].strip("'")] = float(chunk.split('*')[0])
    topics.append(topic)
    
from pprint import pprint
#pprint(top_topics)
df_list=[]

for topic in topics:

    df_= pd.DataFrame([(str(k),v) for k,v in topic.iteritems() ])
    df_.sort_values([1], ascending=False)
    
    df_list.append( df_[0])

df = pd.concat(df_list, axis = 1)
df.columns=[ [ 'Topic'+str(i) for i in range(len(df_list))]]


In [5]:
#df_text=pd.read_csv('../DataFiles/sentences_withPersonName_NYC_listing.csv')
#df_text=df_text[df_text.year>2009]
#df_=pd.DataFrame(df_text.groupby(['listing_id']).year.count())
#indices=df_[df_.year==7].index.tolist()
#df_text=df_text[df_text['listing_id'].isin(indices)]


In [6]:

num_topics= model.num_topics
for i in range(num_topics):
    df_text['topic'+str(i)]=0.0

for i in range(len(corpus)):
    topic_dist=model.get_document_topics(corpus[i])
    if i % 1000 ==0:print i,'/',len(corpus)

    for j in range(len(topic_dist)):
        topic_no=topic_dist[j][0]
        
        df_text.set_value(i,'topic'+str(j) , topic_dist[j][1])
        
df_text.to_csv('../DataFiles/sentences_withPersonName_NYC_listing_topics.csv')       
df_text

NameError: name 'df_text' is not defined

In [ ]:
topic_dist
['year']+ ['topic'+str(i) for i in range(10)]

In [ ]:
df_year=df_text[['year']+ ['topic'+str(i) for i in range(10)]]
df_year=df_year.groupby('year').mean()
df_year.to_csv('../DataFiles/topicOverTimeNYC.csv')

# Chanages of contribution to each topic over time

In [ ]:
df_year=pd.read_csv('../DataFiles/topicOverTimeNYC.csv')
 
params = {'legend.fontsize': 8,}
plt.rcParams['figure.figsize'] = 12,8
f, axarr = plt.subplots(4,3)
plt.subplots_adjust(hspace=1)

plt.suptitle('Trends of Topics 2010-2016  in NYC\
             \n(percentage of mentioning of each aspect in whole reviews)',fontsize=16)
k=0
x=range(2010,2017)
for i in range(0,4):
    for j in range(0,3):

        sns.set_style("darkgrid")
        if k>9: break
        plt.rcParams.update(params)
        y=df_year['topic'+str(k)]

        axarr[i, j].plot(x, y)
        axarr[i, j].set_title('topic'+str(k))
      #  axarr[i, j].set_ylim(0,1)
        axarr[i, j].get_xaxis().get_major_formatter().set_useOffset(False)
        axarr[i, j].yaxis.set_major_formatter(mtick.FormatStrFormatter('%.2f'))
        for tick in axarr[i, j].get_xaxis().get_major_ticks():

            tick.label.set_fontsize(8) 
#            tick.label.set_rotation('vertical')

        k+=1